In [1]:
def memoize(func):
    """
    Store the results of the decorated function for fast lookup
    """
    # Store results in a dict that maps arguments to results
    cache = {}

    # Define the wrapper function to return.
    def wrapper(*args, **kwargs):
        # Define a hashable key for 'kwargs'.
        kwargs_key = tuple(sorted(kwargs.items()))
        # If these arguments haven't been seen before,
        if (args, kwargs_key) not in cache:
            # Call func() and store the result.
            cache[(args, kwargs_key)] = func(*args, **kwargs)
        return cache[(args, kwargs_key)]

    return wrapper


In [2]:
def parent_func():
    def child_func(x =100):
        print(x * 1.08)
    return child_func

In [3]:
output = parent_func()

In [4]:
output()

108.0


In [5]:
output(23)

24.840000000000003


In [7]:
print(output())

108.0
None


In [27]:
def counter(func):
  """This is a docstring for parent_func, counter"""
  def wrapper(*args, **kwargs):
    """This is a docstring for child_func, wrapper"""
    wrapper.count += 1
    # Call the function being decorated and return the result
    return func(*args, **kwargs)
  # Set count to 0 to initialize call count for each new decorated function
  wrapper.count = 0
  # Return the new decorated function
  return wrapper

In [ ]:

# Decorate foo() with the counter() decorator
@counter
def foo():
  print('calling foo()')
  
foo()
foo()

print('foo() was called {} times.'.format(foo.count))

In [28]:
def foo():
  """This is a docstring for simple foo, outside"""
  print('calling foo()')

In [29]:
foo = counter(foo)

In [11]:
foo.__closure__

(<cell at 0x1102eb400: function object at 0x1102e65c0>,
 <cell at 0x1102e8280: function object at 0x1102e6340>)

In [30]:
doo = foo.__closure__[0].cell_contents

In [31]:
print(doo.__code__.co_varnames)        # Local variable names
print(doo.__code__.co_argcount)        # Number of positional arguments
print(doo.__code__.co_consts)        # Constants in the function
print(doo.__code__.co_names)        # All names used (functions, vars, etc.)
print(doo.__code__.co_filename)        # Source file name
print(doo.__code__.co_firstlineno)     # Line number where the function starts


()
0
('This is a docstring for simple foo, outside', 'calling foo()', None)
('print',)
/var/folders/n0/9wldyr854817hc82h7w85z6c0000gn/T/ipykernel_9012/1333178602.py
1


In [32]:
doo.__doc__

'This is a docstring for simple foo, outside'

In [ ]:
#I could not understand memoize wraper. I am not understanding counter wrapper. Is there someting wrong with my brain? What the hell am I stuck. Let me try to understand this.

def counter(func):
    """This is a docstring for parent_func, counter"""
    def wrapper(*args, **kwargs):
        """This is a docstring for child_func, wrapper"""
        wrapper.count += 1
        # What is wrapper.count? How is a function able to have a count and how is it being defined within t
        # Call the function being decorated and return the result
        return func(*args, **kwargs)
    
    wrapper.count = 0
    # The wrapper count 
    # Return the new decorated function
    return wrapper